# 如何使用传统的LangChain代理（AgentExecutor）

:::info 预备知识

本指南假定您熟悉以下概念：

- [工具](/docs/concepts/tools)

:::

语言模型本身无法执行操作 - 它们只是输出文本。
代理是一种系统，它使用LLM作为推理引擎来确定要执行哪些操作以及这些操作的输入应该是什么。
这些操作的结果随后可以反馈给代理，代理将确定是否需要执行更多操作，或者是否可以结束。

在本教程中，我们将构建一个可以与多个不同工具交互的代理：一个是本地数据库，另一个是搜索引擎。您可以向这个代理提问，观察它调用工具，并与它进行对话。

:::{.callout-important}
本节将介绍使用LangChain代理进行构建。LangChain代理适合入门，但超过一定阶段后，您可能需要它们无法提供的灵活性和控制力。对于使用更高级的代理，我们建议查看[LangGraph](https://langchain-ai.github.io/langgraphjs)。
:::

## 概念

我们将介绍的概念包括：
- 使用[语言模型](/docs/concepts/chat_models)，特别是它们的工具调用能力
- 创建一个[检索器](/docs/concepts/retrievers)，以向我们的代理暴露特定信息
- 使用搜索[工具](/docs/concepts/tools)在线查找内容
- [`聊天历史`](/docs/concepts/chat_history)，这允许聊天机器人“记住”过去的交互，并在回应后续问题时考虑这些信息。
- 使用[LangSmith](/docs/concepts/#langsmith)调试和追踪您的应用程序

## 准备工作

### Jupyter Notebook

本指南（以及文档中的大多数其他指南）使用[Jupyter笔记本](https://jupyter.org/)，并假定读者也使用。Jupyter笔记本非常适合学习如何操作LLM系统，因为很多时候可能会出错（意外输出、API中断等），而在交互式环境中进行指南是更好地理解它们的好方法。

这个及其他教程可能最适合在Jupyter笔记本中运行。有关安装说明，请参见[此处](https://jupyter.org/install)。

### 安装

要安装LangChain（以及web加载器的`cheerio`），运行：

```{=mdx}
import Npm2Yarn from '@theme/Npm2Yarn';

<Npm2Yarn>
  langchain @langchain/core cheerio
</Npm2Yarn>
```

更多详细信息，请参阅我们的[安装指南](/docs/how_to/installation/)。

### LangSmith

使用LangChain构建的许多应用程序将包含多个步骤，涉及多次调用LLM。
随着这些应用程序变得越来越复杂，能够检查您的链或代理内部到底发生了什么变得至关重要。
最好的方法是使用[LangSmith](https://smith.langchain.com)。

在上方链接注册后，请确保设置您的环境变量以开始记录追踪：

```shell
export LANGSMITH_TRACING="true"
export LANGSMITH_API_KEY="..."

# 如果您不在无服务器环境中，请减少追踪延迟
# export LANGCHAIN_CALLBACKS_BACKGROUND=true
```


## 定义工具

我们首先需要创建我们想要使用的工具。我们将使用两个工具：[Tavily](/docs/integrations/tools/tavily_search)（用于在线搜索），然后是我们将创建的本地索引的检索器

### [Tavily](/docs/integrations/tools/tavily_search)

LangChain中有一个内置工具，可以方便地将Tavily搜索引擎作为工具使用。
请注意，这需要一个API密钥——他们有免费的层级，但如果您没有或者不想创建一个，您可以随时跳过此步骤。

一旦您创建了API密钥，您需要将其导出为：

```bash
export TAVILY_API_KEY="..."
```

In [1]:
import "cheerio"; // This is required in notebooks to use the `CheerioWebBaseLoader`
import { TavilySearchResults } from "@langchain/community/tools/tavily_search"

const search = new TavilySearchResults({
  maxResults: 2
});

await search.invoke("what is the weather in SF")

`[{"title":"Weather in San Francisco","url":"https://www.weatherapi.com/","content":"{'location': {'n`... 1358 more characters

### 检索器

我们还将基于我们自己的某些数据创建一个检索器。有关每个步骤的更深入解释，请参阅[本教程](/docs/tutorials/rag)。

In [2]:
import { CheerioWebBaseLoader } from "@langchain/community/document_loaders/web/cheerio";
import { MemoryVectorStore } from "langchain/vectorstores/memory";
import { OpenAIEmbeddings } from "@langchain/openai";
import { RecursiveCharacterTextSplitter } from "@langchain/textsplitters";

const loader = new CheerioWebBaseLoader("https://docs.smith.langchain.com/overview");
const docs = await loader.load();
const splitter = new RecursiveCharacterTextSplitter(
  {
    chunkSize: 1000,
    chunkOverlap: 200
  }
);
const documents = await splitter.splitDocuments(docs);
const vectorStore = await MemoryVectorStore.fromDocuments(documents, new OpenAIEmbeddings());
const retriever = vectorStore.asRetriever();

(await retriever.invoke("how to upload a dataset"))[0];

Document {
  pageContent: 'description="A sample dataset in LangSmith.")client.create_examples(    inputs=[        {"postfix": '... 891 more characters,
  metadata: {
    source: "https://docs.smith.langchain.com/overview",
    loc: { lines: { from: 4, to: 4 } }
  },
  id: undefined
}

现在我们已经填充了将要进行检索的索引，可以轻松地将其转换为一个工具（代理正确使用所需的格式）

In [7]:
import { z } from "zod";
import { tool } from "@langchain/core/tools";

const retrieverTool = tool(async ({ input }, config) => {
  const docs = await retriever.invoke(input, config);
  return docs.map((doc) => doc.pageContent).join("\n\n");
}, {
  name: "langsmith_search",
  description:
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
  schema: z.object({
    input: z.string()
  }),
});

### 工具

既然我们已经创建了这两个对象，我们现在可以创建一个工具列表，供后续使用。

In [8]:
const tools = [search, retrieverTool];

## 使用语言模型

接下来，让我们学习如何通过调用工具来使用语言模型。LangChain 支持许多不同的语言模型，你可以根据需要进行替换——在下方选择你要使用的模型！

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs openaiParams={`{ model: "gpt-4" }`} />
```

你可以通过传入消息列表来调用语言模型。默认情况下，响应是一个 `content` 字符串。

In [9]:
// @lc-docs-hide-cell

import { ChatOpenAI } from "@langchain/openai";

const model = new ChatOpenAI({ model: "gpt-4o-mini", temperature: 0 })

In [10]:
const response = await model.invoke([{
  role: "user",
  content: "hi!"
}]);

response.content;

"Hello! How can I assist you today?"

我们现在可以看看如何让此模型能够调用工具。为了实现这一点，我们使用 `.bind` 方法，使语言模型了解这些工具

In [11]:
const modelWithTools = model.bindTools(tools);

现在我们可以调用模型了。让我们首先用一条普通消息调用它，看看它是如何响应的。我们可以同时查看`content`字段和`tool_calls`字段。

In [12]:
const responseWithTools = await modelWithTools.invoke([{
  role: "user",
  content: "Hi!"
}])

console.log(`Content: ${responseWithTools.content}`)
console.log(`Tool calls: ${responseWithTools.tool_calls}`)

Content: Hello! How can I assist you today?
Tool calls: 


现在，让我们尝试用一些需要调用工具的输入来调用它。

In [13]:
const responseWithToolCalls = await modelWithTools.invoke([{
  role: "user",
  content: "What's the weather in SF?"
}])

console.log(`Content: ${responseWithToolCalls.content}`)
console.log(`Tool calls: ${JSON.stringify(responseWithToolCalls.tool_calls, null, 2)}`)

Content: 
Tool calls: [
  {
    "name": "tavily_search_results_json",
    "args": {
      "input": "current weather in San Francisco"
    },
    "type": "tool_call",
    "id": "call_gtJ5rrjXswO8EIvePrxyGQbR"
  }
]


我们可以看到现在没有内容，但有一个工具调用！它要求我们调用 Tavily 搜索工具。

这还没有真正调用该工具——它只是告诉我们应该调用。为了实际调用它，我们需要创建我们的代理。

## 创建代理

既然我们已经定义了工具和LLM，现在可以创建代理了。我们将使用一个工具调用代理——有关此类代理以及其他选项的更多信息，请参阅[此指南](/docs/concepts/agents/)。

我们可以首先选择用于引导代理的提示词：

In [14]:
import { ChatPromptTemplate } from "@langchain/core/prompts";

const prompt = ChatPromptTemplate.fromMessages([
  ["system", "You are a helpful assistant"],
  ["placeholder", "{chat_history}"],
  ["human", "{input}"],
  ["placeholder", "{agent_scratchpad}"],
]);

console.log(prompt.promptMessages);

[
  SystemMessagePromptTemplate {
    lc_serializable: true,
    lc_kwargs: {
      prompt: PromptTemplate {
        lc_serializable: true,
        lc_kwargs: {
          inputVariables: [],
          templateFormat: "f-string",
          template: "You are a helpful assistant"
        },
        lc_runnable: true,
        name: undefined,
        lc_namespace: [ "langchain_core", "prompts", "prompt" ],
        inputVariables: [],
        outputParser: undefined,
        partialVariables: undefined,
        templateFormat: "f-string",
        template: "You are a helpful assistant",
        validateTemplate: true,
        additionalContentFields: undefined
      }
    },
    lc_runnable: true,
    name: undefined,
    lc_namespace: [ "langchain_core", "prompts", "chat" ],
    inputVariables: [],
    additionalOptions: {},
    prompt: PromptTemplate {
      lc_serializable: true,
      lc_kwargs: {
        inputVariables: [],
        templateFormat: "f-string",
        template: "You ar

现在，我们可以使用LLM、提示词和工具来初始化代理。代理负责接收输入并决定采取什么操作。关键的是，代理不会执行这些操作——这些操作是由AgentExecutor（下一步）完成的。关于如何理解这些组件的更多信息，请参阅我们的[概念指南](/docs/concepts/agents)。

请注意，我们传递的是`model`而不是`modelWithTools`。这是因为`createToolCallingAgent`将在底层为我们调用`.bind`方法。

In [15]:
import { createToolCallingAgent } from "langchain/agents";

const agent = await createToolCallingAgent({ llm: model, tools, prompt })

最后，我们将代理（大脑）与AgentExecutor中的工具结合在一起（它将重复调用代理并执行工具）。

In [16]:
import { AgentExecutor } from "langchain/agents";

const agentExecutor = new AgentExecutor({
  agent,
  tools
})

## 运行智能体

现在我们可以在一些查询上运行该智能体！请注意，目前这些查询都是**无状态**的（它不会记住之前的交互）。

首先，我们来看看在不需要调用工具的情况下它是如何响应的：

In [17]:
await agentExecutor.invoke({ input: "hi!" })

{ input: "hi!", output: "Hello! How can I assist you today?" }

为了准确了解幕后发生的事情（并确保它没有调用工具），我们可以查看[LangSmith 跟踪](https://smith.langchain.com/public/b8051e80-14fd-4931-be0f-6416280bc500/r)

现在我们尝试在一个应该调用检索器的示例上运行它

In [18]:
await agentExecutor.invoke({ input: "how can langsmith help with testing?" })

{
  input: "how can langsmith help with testing?",
  output: "LangSmith can assist with testing in several ways, particularly for applications built using large l"... 1474 more characters
}

让我们看一下 [LangSmith 追踪](https://smith.langchain.com/public/35bd4f0f-aa2f-4ac2-b9a9-89ce0ca306ca/r)，以确认它是否真的调用了该工具。

现在让我们尝试一个需要调用搜索工具的例子：

In [19]:
await agentExecutor.invoke({ input: "whats the weather in sf?" })

{
  input: "whats the weather in sf?",
  output: "The current weather in San Francisco is as follows:\n" +
    "\n" +
    "- **Temperature**: 15.6°C (60.1°F)\n" +
    "- **Conditio"... 303 more characters
}

我们可以查看 [LangSmith 跟踪](https://smith.langchain.com/public/dfde6f46-0e7b-4dfe-813c-87d7bfb2ade5/r)，以确保它有效地调用了搜索工具。

## 添加内存

如前所述，此代理是无状态的。这意味着它不会记住之前的交互。为了赋予它记忆功能，我们需要传入之前的 `chat_history`。

**注意**：由于我们使用的提示，输入变量需要称为 `chat_history`。如果我们使用不同的提示，可以更改变量名。

In [20]:
// Here we pass in an empty list of messages for chat_history because it is the first message in the chat
await agentExecutor.invoke({ input: "hi! my name is bob", chat_history: [] })

{
  input: "hi! my name is bob",
  chat_history: [],
  output: "Hello Bob! How can I assist you today?"
}

In [21]:
await agentExecutor.invoke(
  {
    chat_history: [
      { role: "user", content: "hi! my name is bob" },
      { role: "assistant", content: "Hello Bob! How can I assist you today?" },
    ],
    input: "what's my name?",
  }
)

{
  chat_history: [
    { role: "user", content: "hi! my name is bob" },
    {
      role: "assistant",
      content: "Hello Bob! How can I assist you today?"
    }
  ],
  input: "what's my name?",
  output: "Your name is Bob. How can I help you today, Bob?"
}

如果我们想自动跟踪这些消息，可以将其包装在RunnableWithMessageHistory中。

由于我们有多个输入，我们需要指定两件事情：

- `inputMessagesKey`：用于添加到对话历史记录的输入键。
- `historyMessagesKey`：将加载的消息添加到的目标键。

有关如何使用此功能的更多信息，请参阅[此指南](/docs/how_to/message_history)。

In [22]:
import { ChatMessageHistory } from "@langchain/community/stores/message/in_memory";
import { BaseChatMessageHistory } from "@langchain/core/chat_history";
import { RunnableWithMessageHistory } from "@langchain/core/runnables";

const store = {};

function getMessageHistory(sessionId: string): BaseChatMessageHistory {
  if (!(sessionId in store)) {
    store[sessionId] = new ChatMessageHistory();
  }
  return store[sessionId];
}

const agentWithChatHistory = new RunnableWithMessageHistory({
  runnable: agentExecutor,
  getMessageHistory,
  inputMessagesKey: "input",
  historyMessagesKey: "chat_history",
})

await agentWithChatHistory.invoke(
  { input: "hi! I'm bob" },
  { configurable: { sessionId: "<foo>" }},
)

{
  input: "hi! I'm bob",
  chat_history: [
    HumanMessage {
      "content": "hi! I'm bob",
      "additional_kwargs": {},
      "response_metadata": {}
    },
    AIMessage {
      "content": "Hello Bob! How can I assist you today?",
      "additional_kwargs": {},
      "response_metadata": {},
      "tool_calls": [],
      "invalid_tool_calls": []
    }
  ],
  output: "Hello Bob! How can I assist you today?"
}

In [23]:
await agentWithChatHistory.invoke(
  { input: "what's my name?" },
  { configurable: { sessionId: "<foo>" }},
)

{
  input: "what's my name?",
  chat_history: [
    HumanMessage {
      "content": "hi! I'm bob",
      "additional_kwargs": {},
      "response_metadata": {}
    },
    AIMessage {
      "content": "Hello Bob! How can I assist you today?",
      "additional_kwargs": {},
      "response_metadata": {},
      "tool_calls": [],
      "invalid_tool_calls": []
    },
    HumanMessage {
      "content": "what's my name?",
      "additional_kwargs": {},
      "response_metadata": {}
    },
    AIMessage {
      "content": "Your name is Bob! How can I help you today, Bob?",
      "additional_kwargs": {},
      "response_metadata": {},
      "tool_calls": [],
      "invalid_tool_calls": []
    }
  ],
  output: "Your name is Bob! How can I help you today, Bob?"
}

示例 LangSmith 追踪：https://smith.langchain.com/public/98c8d162-60ae-4493-aa9f-992d87bd0429/r

## 下一步

本快速入门到此结束！我们介绍了如何创建一个简单的智能体（agent）。智能体是一个复杂的主题，还有很多内容需要学习！

:::{.callout-important}
本节介绍了使用 LangChain 智能体进行构建。LangChain 智能体适合入门使用，但在达到一定阶段后，你可能会需要它们无法提供的灵活性和控制能力。对于更高级的智能体开发，我们建议你查看 [LangGraph](https://langchain-ai.github.io/langgraphjs)。

你也可以查看[这份迁移到 LangGraph 的指南](/docs/how_to/migrate_agent)。
:::